In [1]:
import os
import pandas as pd
import dendropy
import csv
from Bio import SeqIO
import numpy as np
import json
import egglib

Filters already applied when generating  alignments:

    -drop sequences for which a strain has CNV in the gene
    -drop sequences with mid-sequence stop codons
    -drop alignments with valid sequences from fewer than 75% of strains

In [2]:
#get list of AZ strains

strains = pd.read_csv('../isolate_and_pop_info/isolate_and_population_info.csv')
AZ_strains = strains[strains['Population']=='AZ']['Isolate ID'].tolist()
TXMXSA_strains = strains[strains['Population']=='TXMXSA']['Isolate ID'].tolist()

### Calculate Pi for AZ population

In [3]:
def pi(d):
    return dendropy.calculate.popgenstat.nucleotide_diversity(d)

In [4]:
def filter_strains(file, gap_max=0, strain_count_min=0):

    seq_dict = {}
    
    for record in SeqIO.parse(file, 'fasta'):
        
        strain=record.id
        seq = str(record.seq)
        if (seq.count('-') + seq.count('N') + seq.count('n')) / len(seq) > gap_max:
            continue
        seq_dict[strain] = seq

    if len(seq_dict) < strain_count_min:
        return 

    return seq_dict

In [8]:
strain_count_min=.75*len(AZ_strains)
gap_max=0.05
fasta_dir = '../sequence_data/alignments/AZ_gene_fastas_aligned/'

out_dict = {}
    
for f in sorted(os.listdir(fasta_dir)):
    
    gene = f.split('.')[0]
    strain_dict = filter_strains(fasta_dir+f, gap_max, strain_count_min)
    
    if strain_dict:
        
        d = dendropy.DnaCharacterMatrix.from_dict(strain_dict)
        pi_val = pi(d)
        out_dict[gene] = [pi_val]

    

In [7]:
pi_AZ_df = pd.DataFrame.from_dict(out_dict, orient='index')
pi_AZ_df.columns=['pi_AZ']
pi_AZ_df.head()

,pi_AZ
D8B26_000001,0.013427
D8B26_000002,0.004111
D8B26_000003,0.006430
D8B26_000004,0.003481
D8B26_000007,0.000145


### Calculate Pi for TX/MX/SA population

In [11]:
strain_count_min=.75*len(TXMXSA_strains)
gap_max=0.05
fasta_dir = '../sequence_data/alignments/TXMXSA_gene_fastas_aligned/'

out_dict = {}
    
for f in sorted(os.listdir(fasta_dir)):
    
    gene = f.split('.')[0]
    strain_dict = filter_strains(fasta_dir+f, gap_max, strain_count_min)
    
    if strain_dict:
        
        d = dendropy.DnaCharacterMatrix.from_dict(strain_dict)
        pi_val = pi(d)
        out_dict[gene] = [pi_val]
    

In [12]:
pi_TXMXSA_df = pd.DataFrame.from_dict(out_dict, orient='index')
pi_TXMXSA_df.columns=['pi_TXMXSA']
pi_TXMXSA_df.head()

,pi_TXMXSA
D8B26_000009,0.002158
D8B26_000010,0.002826
D8B26_000011,0.003799
D8B26_000013,0.001642
D8B26_000014,0.001641


### Calc DXY between AZ and TX/MX/SA population

In [13]:
good_nucs = 'ACGTactg'

def dxy(pop1_seqs, pop2_seqs):

    pop1_strain_count, pop2_strain_count = len(pop1_seqs), len(pop2_seqs)
    dxy = 0
    for pop1_seq in pop1_seqs:
        for pop2_seq in pop2_seqs:
            for i in range(len(pop1_seq)):
                if pop2_seq[i] != pop1_seq[i] and pop2_seq[i] in good_nucs and pop1_seq[i] in good_nucs:
                    dxy += 1

    dxy = dxy / (pop1_strain_count * pop2_strain_count * len(pop1_seqs[0]))
    return dxy

In [14]:
def filter_strains(file, gene, gap_max=0, pop1_strain_count_min=0, pop2_strain_count_min=0):

    pop1_seq_dict, pop2_seq_dict = {}, {}  

    for record in SeqIO.parse(file, 'fasta'):
        
        strain=record.description.split('_variants.')[0]
        seq = str(record.seq)
        
        if strain in AZ_strains:
            if (seq.count('-') + seq.count('N') + seq.count('n')) / len(seq) > gap_max:
                continue
            pop1_seq_dict[strain] = seq
        
        elif strain in TXMXSA_strains:
            if (seq.count('-') + seq.count('N') + seq.count('n')) / len(seq) > gap_max:
                continue
            pop2_seq_dict[strain] = seq

    if len(pop1_seq_dict) < pop1_strain_count_min or len(pop2_seq_dict) < pop2_strain_count_min:
        return

    return pop1_seq_dict, pop2_seq_dict

In [15]:
aligned_fastas = '../sequence_data/alignments/AZ_TXMXSA_gene_fastas_aligned/'
dxy_dict = {}

gap_max=0.05
AZ_strain_count_min=.75*len(AZ_strains)
TXMXSA_strain_count_min=.75*len(TXMXSA_strains)

for f in sorted(os.listdir(aligned_fastas)):
    
    gene = f.split('.')[0]
    seq_dict = filter_strains(aligned_fastas+f, gene, gap_max=gap_max,  
                              pop1_strain_count_min=AZ_strain_count_min, 
                              pop2_strain_count_min=TXMXSA_strain_count_min)

    if seq_dict:
        AZ_seq_dict, TXMXSA_seq_dict = seq_dict
        dxy_gene = dxy(list(AZ_seq_dict.values()), list(TXMXSA_seq_dict.values()))
        dxy_dict[gene] = dxy_gene

In [16]:
dxy_df = pd.DataFrame.from_dict(dxy_dict, orient='index')
dxy_df.columns=['DXY']
dxy_df.head()

,DXY
D8B26_000009,0.002055
D8B26_000010,0.002184
D8B26_000011,0.003887
D8B26_000013,0.001649
D8B26_000014,0.002098


### Calculate PnPs for AZ population

PnPs = (N/NSites)/(S/SSites)

formula from: https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1008223#pgen.1008223.ref095

In [17]:
pn_ps_dict = {}

codon_alignments = '../sequence_data/alignments/AZ_codon_aligned/'

for f in sorted(os.listdir(codon_alignments)):

    gene = f.replace('.pal2nal.fasta', '')

    aln = egglib.io.from_fasta(codon_alignments+f,alphabet=egglib.alphabets.DNA)

    codons = egglib.tools.to_codons(aln)
    cd = egglib.stats.CodingDiversity(codons)

    N, Nsites = cd.num_pol_NS,cd.num_sites_NS
    S, Ssites = cd.num_pol_S,cd.num_sites_S

    try:
        pnps = (N/Nsites)/(S/Ssites)

    except ZeroDivisionError:
        pnps = np.inf

    pn_ps_dict[gene] = [pnps]


In [19]:
pnps_df = pd.DataFrame.from_dict(pn_ps_dict, orient='index')
pnps_df.columns = ['PnPs']
pnps_df.head()

,PnPs
D8B26_000001,0.907040
D8B26_000002,0.430434
D8B26_000003,0.524033
D8B26_000004,0.270093
D8B26_000006,0.652013


### Combine stats

In [23]:
merged = pi_AZ_df.merge(pi_TXMXSA_df, left_index=True, right_index=True, how='outer')
merged = merged.merge(dxy_df, left_index=True, right_index=True, how='outer')
merged = merged.merge(pnps_df, left_index=True, right_index=True, how='outer')
merged.head()

,pi_AZ,pi_TXMXSA,DXY,PnPs
D8B26_000001,0.013427,NaN,NaN,0.907040
D8B26_000002,0.004111,NaN,NaN,0.430434
D8B26_000003,0.006430,NaN,NaN,0.524033
D8B26_000004,0.003481,NaN,NaN,0.270093
D8B26_000006,NaN,NaN,NaN,0.652013


In [24]:
merged.to_csv('../tables/tableS1_sequence_stats_by_gene.csv')